In [1]:
import os
import numpy as np
import sys
import time
import _init_paths
from core.config import cfg, merge_cfg_from_file, merge_cfg_from_list, assert_and_infer_cfg
from core.test_engine import run_inference, get_inference_dataset
from datasets.json_dataset import JsonDataset
import pickle
import json
from utils.boxes import xywh_to_xyxy, bbox_overlaps
import ipdb

In [2]:
cfg.TEST.DATASETS = ('vcoco_test',)
cfg.MODEL.NUM_CLASSES = 81
cfg.TEST.PRECOMPUTED_PROPOSALS = False
cfg.MODEL.VCOCO_ON = True

In [6]:
ican_res_file = '../ican/ican_vcoco_det_res.json'
vcoco_det_res = json.load(open(ican_res_file, 'r'))

In [20]:
file_path = '../Outputs/e2e_faster_rcnn_R-50-FPN_1x_vcoco/Nov26-09-29-09_wanbo_node35_step/test/bbox_vcoco_test_results.json'
# file_path = '../data/test/bbox_vcoco_test_results.json'
vcoco_det_res = json.load(open(file_path, 'rb'))

In [7]:
dataset_name, proposal_file = get_inference_dataset(0, is_parent=False)
dataset = JsonDataset(dataset_name)
vcocodb = dataset.get_roidb(gt=True)

loading annotations into memory...
Done (t=1.26s)
creating index...
index created!
loading vcoco annotations...


In [9]:
len(vcoco_det_res)

119061

In [10]:
results = {}
for det in vcoco_det_res:
    im_id = det['image_id']
    tmp = results.setdefault(im_id, [])
    tmp.append(np.concatenate([det['bbox'], [det['score'], det['category_id']]]))
    results[im_id] = tmp

for k,v in results.items():
    v = np.array(v).astype(np.float32)
    trans = xywh_to_xyxy(v[:, :4])
    v[:, :4] = trans
    results[k] = v

In [19]:
no_gt_image_num = 0
no_im_id = 0
total_num = [0, 0]
recall_num = [0, 0]

for gt in vcocodb:
#     gt = vcocodb[1]
    im_id = gt['id']

    gt_boxes = gt['boxes'].astype(np.float32)
    gt_cls = gt['gt_classes']
    gt_actions = gt['gt_actions']
    gt_role_inds = gt['gt_role_id']
    gt_human_inds, gt_act_inds, gt_r_ids = np.where(gt_role_inds>-1)

    if len(gt_human_inds) == 0:
        no_gt_image_num += 1
        continue

    gt_obj_inds = gt_role_inds[gt_human_inds, gt_act_inds, gt_r_ids]
    gt_human_inds = np.unique(gt_human_inds)
    gt_obj_inds = np.unique(gt_obj_inds)
    gt_human_box = gt_boxes[gt_human_inds]
    gt_obj_box = gt_boxes[gt_obj_inds]

    total_num[0] += len(gt_human_box)
    total_num[1] += len(gt_obj_box)

    if results.get(im_id) is None:
        no_im_id += 1
        continue
    pred_res = results[im_id]
    pred_boxes = pred_res[:, :4]
    pred_scores = pred_res[:, -2]
    pred_cls = pred_res[:, -1].astype(np.int)

    pred_obj_ind = np.where(pred_cls>1)
    if len(pred_obj_ind[0]) == 0:
        continue
    pred_obj_box = pred_boxes[pred_obj_ind]
    pred_obj_score = pred_scores[pred_obj_ind]

    valid = np.where(pred_obj_score>=0.4)
    if len(valid[0]) > 0:
        valid_obj_box = pred_obj_box[valid]
        obj_overlap = bbox_overlaps(gt_obj_box, valid_obj_box)
    #     obj_overlap = bbox_overlaps(gt_obj_box, pred_obj_box)
        try:
            obj_max_overlap = obj_overlap.max(1)
        except:
            ipdb.set_trace()
        recall_num[1] += np.sum(obj_max_overlap>=0.5)

    pred_human_ind = np.where(pred_cls==1)
    if len(pred_human_ind[0]) == 0:
        continue
    pred_human_box = pred_boxes[pred_human_ind]
    pred_human_scores = pred_scores[pred_human_ind]
    valid = np.where(pred_human_scores>=0.5)
    if len(valid[0]) > 0:
        valid_human_box = pred_human_box[valid]
        human_overlap = bbox_overlaps(gt_human_box, valid_human_box)
    #     human_overlap = bbox_overlaps(gt_human_box, pred_human_box)
        human_max_overlap = human_overlap.max(1)
        recall_num[0] += np.sum(human_max_overlap>=0.5)
    

In [20]:
total_num

[6287, 7020]

In [21]:
recall_num

[6106, 5699]

In [22]:
5699/7020

0.8118233618233618

In [23]:
6106/6287

0.9712104342293622

In [12]:
no_gt_image_num

407

In [17]:
type(results[165])

numpy.ndarray

In [4]:
save_path = '../data/cache/vcoco_test_precomp_boxes.json'
for k,v in results.items():
    results[k] = v.tolist()
    
with open(save_path, 'w') as f:
    json.dump(results, f)

In [25]:
5911/7072

0.8358314479638009

In [26]:
len(results)

4946